In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import glob
import sys 
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm

In [2]:
ACTIONS = [
  'read_comment',
  'like',
  'click_avatar',
  'forward',
  'favorite',
  'comment',
  'follow'
]

In [3]:
feeds = {}
def cache_feed():
  df = pd.read_csv('../input/feed_info.csv')
  df = df.fillna(-1)
  for row in tqdm(df.itertuples(), total=len(df), desc='feed_info'):
    row = row._asdict()
    feeds[row['feedid']] = row
cache_feed()

feed_info:   0%|          | 0/106444 [00:00<?, ?it/s]

In [4]:
def num_actions(row):
  cnt = 0
  for action in ACTIONS:
    if row[action] > 0:
      cnt += 1
  return cnt

In [5]:
with gezi.Timer('read_actions csv', True):
  d = pd.read_csv('../input/user_action.csv')
  d['version'] = 2
  try:
    d1 = pd.read_csv('../input/v1/user_action.csv')
    d1['version'] = 1
    d = pd.concat([d, d1])
  except Exception:
    pass
  d.feedid = d.feedid.astype(int)
  d.userid = d.userid.astype(int)
  d.date_ = d.date_.astype(int)

read_actions csv start
read_actions csv duration: 2.83491587638855


In [6]:
d.head()

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,version
0,8,71474,1,1,0,0,1,500,5366,0,0,0,0,2
1,8,73916,1,1,0,0,0,250,1533,0,0,0,0,2
2,8,50282,1,1,0,0,0,750,1302,0,0,0,0,2
3,8,11391,1,1,0,0,1,3750,5191,0,0,0,0,2
4,8,27349,1,1,0,0,0,250,800,0,0,0,0,2


In [7]:
finish_rates = []
stay_rates = []
is_firsts = []
actions = []

d = d.sort_values(['date_'], ascending=True)
m = defaultdict(int)
for row in tqdm(d.itertuples(), total=len(d), desc='user_action'):
  row = row._asdict()
  vtime = min(feeds[row['feedid']]['videoplayseconds'], 60) * 1000 # to ms
  vtime = feeds[row['feedid']]['videoplayseconds'] * 1000 # to ms
  finish_rates.append(row['play'] / vtime)
  stay_rates.append(row['stay'] / vtime)
  userid = row['userid']
  feedid = row['feedid']
  key = f'{userid}_{feedid}'
  m[key] += 1
  is_firsts.append(int(m[key] == 1))
  actions.append(num_actions(row))
d['finish_rate'] = finish_rates
d['stay_rate'] = stay_rates
d['is_first'] = is_firsts
d['actions'] = actions

user_action:   0%|          | 0/7317882 [00:00<?, ?it/s]

In [8]:
# d.head()

In [9]:
# d.describe()

In [10]:
# timer = gezi.Timer('save user_action2.csv')
# d.to_csv('../input/user_action2.csv', index=False)
# timer.print_elapsed()

In [14]:
# timer = gezi.Timer('read user_action2.csv', True)
# d = pd.read_csv('../input/user_action2.csv')
# d.feedid = d.feedid.astype(int)
# d.userid = d.userid.astype(int)
# d.date_ = d.date_.astype(int)
# timer.print_elapsed()
timer = gezi.Timer('save user_action2.feather', True)
d.reset_index().to_feather('../input/user_action2.feather')
timer.print()

save user_action2.feather start
save user_action2.feather duration: 1.6008670330047607


In [15]:
timer = gezi.Timer('read user_action2.feather', True)
pd.read_feather('../input/user_action2.feather')
timer.print()

read user_action2.feather start
read user_action2.feather duration: 0.9051821231842041


In [17]:
for i in tqdm(range(14)):
  day = i + 1
  d[d.date_ == day].reset_index().to_feather(f'../input/user_action2_{day}.feather')

  0%|          | 0/14 [00:00<?, ?it/s]